In [1]:
import numpy as np
from matplotlib import pyplot as plt

<h2>Demonstration of a hadamard gate on single qubit</h2>
<img src='resources/nate/single_hadamard.png' width="200"/>

The hadamard is represented as a 2x2 matrix operator $\frac{1}{\sqrt(2)} \begin{bmatrix}1&1\\1&-1 \end{bmatrix}$

A qubit is provided, initialized at state $\ket{\psi} = \ket{0}$
Hadamard is applied on the qubit, throwing it into superposition $\frac{\ket{0}+\ket{1}}{\sqrt{2}}$

A measurement is then taken, collapsing the qubit to either state $\ket{0}$ or $\ket{1}$ with probability $|\frac{1}{\sqrt{2}}|^{2} = 0.5$ for both states.